In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

In [2]:
import sys
sys.path.insert(1, 'C:/Users/pnknv/Documents/GitHub/yadisk_loader')
from main_utils import *

In [3]:
st_time = datetime.now()
print('Скрипт запущен: ', get_today_date())

Скрипт запущен:  18/04/2023, 20:56:42


In [251]:
# для обращения к препрод базе

In [253]:
def get_config_1(get_only_start_time=False):
    config = configparser.ConfigParser()
    config.read('config.ini')

    ssh_host = config['database']['ssh_host_main']  # переключить на ssh_host_main для работы на прод сервере
    ssh_port = int(config['database']['ssh_port'])
    ssh_username = config['database']['ssh_username']
    ssh_password = config['database']['ssh_password']
    database_username = config['database']['database_username']
    database_password = config['database']['database_password']
    database_name = config['database']['database_name']
    preprod_database_name = config['database']['preprod_database_name']
    localhost = config['database']['localhost']
    localhost_port = int(config['database']['localhost_port'])
    table_name = config['database']['table_name']

    ya_token = config['yandex']['ya_token']
    ya_api = config['yandex']['ya_api']
    ya_link = config['yandex']['ya_link_test']

    start_time = int(config['start_time']['daily_start_hour'])

    if get_only_start_time == False:
        return ssh_host, ssh_port, ssh_username, ssh_password, database_username, database_password, preprod_database_name, \
               localhost, localhost_port, table_name, ya_token, ya_api, ya_link
    else:
        return start_time

# чтение локальных файлов

In [4]:
ilya_c = pd.read_csv('циан 16-04-23 (без дублей) new.csv', delimiter=',', encoding='utf8')

In [5]:
ilya_c.head(5)

,Ссылка,Заголовок,Комнат,Площадь,Этаж,Цена,Цена за м2,Название жк,Адрес,Дата публикации,Тип продавца,Продавец
0,https://kazan.cian.ru/sale/flat/286179179/,"1-комн. кв., 44,7 м², 1/10 этаж",1-комн. кв.,"44,7 м²",1/10 этаж,7700000,172260,NaN,Республика Татарстан;Казань;р-н Кировский;мкр....,"вчера (вчера, 14:40)",Агентство недвижимости,Альтера
1,https://kazan.cian.ru/sale/flat/286178456/,"1-комн. кв., 43,2 м², 3/24 этаж",1-комн. кв.,"43,2 м²",3/24 этаж,7125000,164931,ЖК «Ракета»,Республика Татарстан;Казань;р-н Советский;улиц...,"два дня назад (13 апр, 14:17)",Риелтор,ID 67064013
2,https://kazan.cian.ru/sale/flat/286176086/,"1-комн. кв., 40,8 м², 4/19 этаж",1-комн. кв.,"40,8 м²",4/19 этаж,7500000,183824,ЖК «Золотая середина»,Республика Татарстан;Казань;р-н Ново-Савиновск...,"вчера (14 апр, 10:54)",Агентство недвижимости,Альтера
3,https://kazan.cian.ru/sale/flat/286172402/,"2-комн. кв., 60 м², 4/7 этаж",2-комн. кв.,60 м²,4/7 этаж,7599000,126650,NaN,Республика Татарстан;Казань;р-н Приволжский;мк...,"1 минуту назад (сегодня, 10:29)",Агентство недвижимости,Ключ
4,https://kazan.cian.ru/sale/flat/286167953/,"2-комн. кв., 45 м², 5/12 этаж",2-комн. кв.,45 м²,5/12 этаж,7500000,166667,ЖК «ул. Максимова»,Республика Татарстан;Казань;р-н Авиастроительн...,"два дня назад (13 апр, 11:34)",Агентство недвижимости,ID 63036280


In [6]:
ilya_a = pd.read_csv('16-04-23 (без дублей).csv', delimiter=';', encoding='utf8')

In [7]:
ilya_a.head(5)

,Ссылка,Заголовок,Комнат,Площадь,Этаж,Цена,Цена за м2,Название жк,Район,Улица,Дата публикации,Тип продавца,Продавец,Ссылка на продавца,Завершённых объявлений
0,https://www.avito.ru/tomsk/kvartiry/2-k._kvart...,"2-к. квартира, 45 м², 5/5 эт.",2-к. квартира,45 м²,5/5 эт.,3500000,77778.0,NaN,р-н Октябрьский,"ул. Мичурина, 61Б",4 дня назад,Агентство,Городской центр недвижимости,https://www.avito.ru/user/95e1bbf88375e37753b4...,190
1,https://www.avito.ru/tomsk/kvartiry/3-k._kvart...,"3-к. квартира, 62,3 м², 1/17 эт.",3-к. квартира,"62,3 м²",1/17 эт.,4980000,79936.0,NaN,NaN,"ул. Титова , д. 12",1 день назад,Застройщик,ОАО ТДСК,https://www.avito.ru/user/aa5496618ee23df30923...,500
2,https://www.avito.ru/tomsk/kvartiry/2-k._kvart...,"2-к. квартира, 61,3 м², 6/6 эт.",2-к. квартира,"61,3 м²",6/6 эт.,6009360,98032.0,NaN,NaN,"ул. Василия Кандинского, д. 8",1 день назад,Застройщик,"""СЗ""Карьероуправление""",https://www.avito.ru/user/4187eee51af74e5107d1...,129
3,https://www.avito.ru/tomsk/kvartiry/1-k._kvart...,"1-к. квартира, 34,9 м², 7/9 эт.",1-к. квартира,"34,9 м²",7/9 эт.,3300000,94556.0,NaN,р-н Советский,"Сибирская ул., 104",6 часов назад,Агентство,Перспектива24-Томск,https://www.avito.ru/user/b1f3c879b3fd07d3d782...,NaN
4,https://www.avito.ru/tomsk/kvartiry/1-k._kvart...,"1-к. квартира, 32 м², 9/9 эт.",1-к. квартира,32 м²,9/9 эт.,2430000,75938.0,NaN,р-н Октябрьский,"ул. Мичурина, 6А",4 дня назад,Частное лицо,NaN,NaN,NaN


# процесс импорта

In [259]:
ssh_host, ssh_port, ssh_username, ssh_password, \
database_username, database_password, database_name, \
localhost, localhost_port, table_name, ya_token, ya_api, ya_link = get_config()

In [260]:
sql_server, sql_engine = get_sql_engine(ssh_host, ssh_port, ssh_username, ssh_password, localhost, 
                                        localhost_port, database_username, database_password, database_name)

In [10]:
local_save_dir_avito = create_load_save_dir('avito')
local_save_dir_cian = create_load_save_dir('cian')

In [11]:
handled_files_avito = get_saved_files_names('avito')
handled_files_cian = get_saved_files_names('cian')

In [12]:
files_to_process_avito = ['16-04-23 (без дублей).csv']
files_to_process_cian = ['циан 16-04-23 (без дублей) new.csv']

### cian

In [13]:
# check cian from ilya
for filename in files_to_process_cian:
    # обработка realty циан
    print('обработка файла {} для cian'.format(filename))
    df_cian_realty, file_date, error_file_processing = process_realty(local_save_dir_cian, filename,
                                                                      sql_engine, 'cian')

обработка файла циан 16-04-23 (без дублей) new.csv для cian
добавлен новый район "кварталы 68" в districts функцией district_from_rn_mkrn
добавлен новый район "кварталы 38" в districts функцией district_from_rn_mkrn
добавлен новый район "кварталы 25" в districts функцией district_from_rn_mkrn
Успешно обработан файл циан 16-04-23 (без дублей) new.csv для realty


In [14]:
df_cian_realty.head(2)

,source_id,ad_id,city_id,district_id,type_id,addr,square,floor,house_floors,link,date,status,version,offer_from,status_new,price,price_sqrm
0,2,286179179,12.0,179,3,Республика Татарстан; Казань; р-н Кировский; м...,44.7,1,10,https://kazan.cian.ru/sale/flat/286179179/,2023-04-16,0,292,Агентство недвижимости,1,7700000,172260
1,2,286178456,12.0,362,3,Республика Татарстан; Казань; р-н Советский; у...,43.2,3,24,https://kazan.cian.ru/sale/flat/286178456/,2023-04-16,0,292,Риелтор,1,7125000,164931


In [15]:
exist_ad_id_cian, error_getting_ad_id = get_exist_ad_id(sql_engine, 'cian')
exist_ad_id_cian = exist_ad_id_cian.ad_id.to_list()

In [16]:
cian_realty_exist = df_cian_realty[df_cian_realty.ad_id.isin(exist_ad_id_cian)][list_realty_cols]
cian_realty_new = df_cian_realty[~df_cian_realty.ad_id.isin(exist_ad_id_cian)][list_realty_cols]

In [17]:
print(len(cian_realty_new))
print(len(cian_realty_exist))
cian_realty_new.head(2)

80582
0


,source_id,ad_id,city_id,district_id,type_id,addr,square,floor,house_floors,link,date,status,version,offer_from,status_new
0,2,286179179,12.0,179,3,Республика Татарстан; Казань; р-н Кировский; м...,44.7,1,10,https://kazan.cian.ru/sale/flat/286179179/,2023-04-16,0,292,Агентство недвижимости,1
1,2,286178456,12.0,362,3,Республика Татарстан; Казань; р-н Советский; у...,43.2,3,24,https://kazan.cian.ru/sale/flat/286178456/,2023-04-16,0,292,Риелтор,1


In [18]:
c_1day = pd.read_csv("циан 15-04-23 (без дублей) new.csv", delimiter=',', encoding='utf8')
c_1m = pd.read_csv("циан 16-03-23 (без дублей).csv", delimiter=';', encoding='utf8')
c_1day['ad_id'] = c_1day['Ссылка'].apply(lambda x: x.split('/')[-2])
c_1m['ad_id'] = c_1m['Ссылка'].apply(lambda x: x.split('/')[-2])
c_1day_adids = c_1day.ad_id.unique().tolist()
c_1m_adids = c_1m.ad_id.unique().tolist()
print(len(c_1day_adids))
print(len(c_1m_adids))

81784
64130


In [19]:
print(len(df_cian_realty[df_cian_realty.ad_id.isin(c_1day_adids)][list_realty_cols]))
print(len(df_cian_realty[~df_cian_realty.ad_id.isin(c_1day_adids)][list_realty_cols]))
print(len(df_cian_realty[df_cian_realty.ad_id.isin(c_1m_adids)][list_realty_cols]))
print(len(df_cian_realty[~df_cian_realty.ad_id.isin(c_1m_adids)][list_realty_cols]))

77680
2902
36167
44415


In [20]:
df_cian_realty.to_csv('processed_cian_16.csv')

### avito

In [21]:
# check avito from ilya
for filename in files_to_process_avito:
    # обработка realty циан
    print('обработка файла {} для avito'.format(filename))
    df_avito_realty, file_date, error_file_processing = process_realty(local_save_dir_avito, filename,
                                                                      sql_engine, 'avito')

обработка файла 16-04-23 (без дублей).csv для avito
Успешно обработан файл 16-04-23 (без дублей).csv для realty


In [22]:
print(len(df_avito_realty))
df_avito_realty.head(2)

120188


,source_id,ad_id,city_id,district_id,type_id,addr,square,floor,house_floors,link,date,status,version,offer_from,status_new,Цена,Цена за м2
0,3,2888300632,17,391,4,Томск; р-н Октябрьский; ул. Мичурина; 61Б,45.0,5,5,https://www.avito.ru/tomsk/kvartiry/2-k._kvart...,2023-04-16,0,1,Агентство,1,3500000.0,77778.0
1,3,2882533936,17,None,5,Томск; ул. Титова ; д. 12,62.3,1,17,https://www.avito.ru/tomsk/kvartiry/3-k._kvart...,2023-04-16,0,1,Застройщик,1,4980000.0,79936.0


In [23]:
a_1day = pd.read_csv("15-04-23 (без дублей).csv", delimiter=';', encoding='utf8')
a_1m = pd.read_csv("16-03-23 (без дублей).csv", delimiter=';', encoding='utf8')
a_1day['ad_id'] = a_1day['Ссылка'].apply(lambda x: x.split('_')[-1])
a_1m['ad_id'] = a_1m['Ссылка'].apply(lambda x: x.split('_')[-1])
a_1day_adids = a_1day.ad_id.unique().tolist()
a_1m_adids = a_1m.ad_id.unique().tolist()
print(len(a_1day_adids))
print(len(a_1m_adids))

121829
121053


In [24]:
print(len(df_avito_realty[df_avito_realty.ad_id.isin(a_1day_adids)][list_realty_cols]))
print(len(df_avito_realty[~df_avito_realty.ad_id.isin(a_1day_adids)][list_realty_cols]))
print(len(df_avito_realty[df_avito_realty.ad_id.isin(a_1m_adids)][list_realty_cols]))
print(len(df_avito_realty[~df_avito_realty.ad_id.isin(a_1m_adids)][list_realty_cols]))

114448
5740
75569
44619


In [25]:
df_avito_realty.to_csv('processed_avito_16.csv')

# обновленный функционал для обращения к dadata и записи в иные таблицы

In [250]:
# запросы на добавить поля
# ad_id в dadata_houses
# ALTER TABLE dadata_houses add COLUMN ad_id int(11) DEFAULT NULL;

In [98]:
# ЦИАН:
# скачал ссв
# обработал до реалти
# фильтр по ад_ид
# те, что есть - только апдейт
## те, что нет - запрос к дадата
##  
##
##
#
#
## скачал ссв

In [165]:
# чтение
df_avito_realty = pd.read_csv('processed_avito_16.csv', index_col=0)
df_cian_realty = pd.read_csv('processed_cian_16.csv', index_col=0)

In [166]:
# 500 случайных со стейтом 111
df_avito_realty_sample = df_avito_realty.sample(500, random_state=111)
df_cian_realty_sample = df_cian_realty.sample(500, random_state=111)

In [168]:
# def filter_addr_for_dadata(addr, source):
#     # возвращает строку адреса без района и микрорайона (авито) без региона, района, микрорайона (циан)
#     if source == 'avito':
#         return '; '.join([ad_el for ad_el in addr.split('; ') if not any(del_el in ad_el for del_el in ['мкр', 'р-н', 'микрорайон', 'район', 'Район', 'Микрорайон'])])
#     else: # source == 'cian'
#         ad_noreg = '; '.join(addr.split('; ')[1:])
#         return '; '.join([ad_el for ad_el in ad_noreg.split('; ') if not any(del_el in ad_el for del_el in ['мкр', 'р-н', 'микрорайон', 'район', 'Район', 'Микрорайон'])])

# def dadata_request_1(df, source):
#     st_time = datetime.now()
#     # dadata_credentials
#     token = "f288b25edb6d05b5ceb4d957376104a181c4adee"
#     secret = "9d337ae6b9901a6708802eaca6d7055ce2c64772"
#     dadata = Dadata(token, secret)
#     # create df for dadata_house_loading
#     dh_df = pd.DataFrame(columns=['house_fias_id', 'data', 'geo_lat', 'geo_lon', 'street', 'house', 'qc', 'result', 'qc_geo', 'ad_id'])
#     # dadata request
#     if source == 'avito':
#         for i, row in df[:10].iterrows():
#             d_res = dadata.clean("address", filter_addr_for_dadata(row.addr, 'avito'))
#             dh_df.loc[len(dh_df.index)] = [d_res['house_fias_id'], d_res, d_res['geo_lat'], d_res['geo_lon'], d_res['street'],
#                                            d_res['house'], d_res['qc'], d_res['result'], d_res['qc_geo'], row.ad_id]
#     else: # source == 'cian'
#         for i, row in df[:10].iterrows():
#             d_res = dadata.clean("address", filter_addr_for_dadata(row.addr, 'cian'))
#             dh_df.loc[len(dh_df.index)] = [d_res['house_fias_id'], d_res, d_res['geo_lat'], d_res['geo_lon'], d_res['street'],
#                                            d_res['house'], d_res['qc'], d_res['result'], d_res['qc_geo'], row.ad_id]
#     print('обращение к дадата по {} записям {} заняло'.format(len(df), source), datetime.now() - st_time)
#     return dh_df

In [180]:
def dadata_request(df, source):
    st_time = datetime.now()
    # dadata_credentials
    token = "f288b25edb6d05b5ceb4d957376104a181c4adee"
    secret = "9d337ae6b9901a6708802eaca6d7055ce2c64772"
    dadata = Dadata(token, secret)
    # create df for dadata_house_loading
    dh_df = pd.DataFrame(columns=['house_fias_id', 'data', 'geo_lat', 'geo_lon', 'street', 'house', 'qc', 'result', 'qc_geo', 'ad_id'])
    # dadata request
    if source == 'avito':
        for i, row in df.iterrows():
            d_res = dadata.clean("address", row.addr)
            dh_df.loc[len(dh_df.index)] = [d_res['house_fias_id'], d_res, d_res['geo_lat'], d_res['geo_lon'], d_res['street'],
                                           d_res['house'], d_res['qc'], d_res['result'], d_res['qc_geo'], row.ad_id]
    else: # source == 'cian'
        for i, row in df.iterrows():
            d_res = dadata.clean("address", '; '.join([row.addr.split('; ')[index] for index in [0, 1, -2, -1]]))
            dh_df.loc[len(dh_df.index)] = [d_res['house_fias_id'], d_res, d_res['geo_lat'], d_res['geo_lon'], d_res['street'],
                                           d_res['house'], d_res['qc'], d_res['result'], d_res['qc_geo'], row.ad_id]
    print('обращение к дадата по {} записям {} заняло'.format(len(df), source), datetime.now() - st_time)
    return dh_df

In [96]:
df_dadata_houses_avito = dadata_request(df_avito_realty_sample, 'avito')

обращение к дадата по 500 записям avito заняло 0:02:57.529274


In [110]:
df_dadata_houses_avito.head(2)

,house_fias_id,data,geo_lat,geo_lon,street,house,qc,result,qc_geo,ad_id
0,93dd5940-a82b-4e15-b2f5-e3cbcd65e19e,{'source': 'Казань; с. Усады; Дорожная ул.; 69...,55.6737522,49.2170794,Дорожная,69,1,"Респ Татарстан, Лаишевский р-н, с Усады, ул До...",2,2302244851
1,c0c52465-8b8c-4d26-a027-f39c15051ba0,"{'source': 'Анапа; Анапское ш.; 30', 'result':...",44.898317,37.354429,Анапское,30,0,"Краснодарский край, г Анапа, ш Анапское, д 30",0,2160111672


In [178]:
df_dadata_houses_avito.to_csv('dadata_houses_avito_16_sample.csv')

In [107]:
# авито: 
# запрос к дадате 500 адресов - 3 минуты
# за день приходит 5700 записей по 7 городам
# в итоге время загрузки увеличивается на 5700/500 * 3 = 34 минуты

In [181]:
df_dadata_houses_cian = dadata_request(df_cian_realty_sample, 'cian')

обращение к дадата по 500 записям cian заняло 0:02:28.207289


In [182]:
df_dadata_houses_cian.head(2)

,house_fias_id,data,geo_lat,geo_lon,street,house,qc,result,qc_geo,ad_id
0,7c695360-8941-45a7-b8d3-ba8be80d23ae,{'source': 'Республика Татарстан; Казань; улиц...,55.7616437,49.1949612,Родины,26Б,0,"г Казань, ул Родины, д 26Б",1,281191356
1,70098c0c-6a32-4319-81ae-dbeb6ac0dbb0,{'source': 'Краснодарский край; Краснодар; Вос...,45.069881,39.024136,Восточно-Кругликовская,65,0,"г Краснодар, ул Восточно-Кругликовская, д 65",0,284975855


In [184]:
# test_cian = pd.merge(df_dadata_houses_cian[df_dadata_houses_cian.house_fias_id.isna()], df_cian_realty_sample[['ad_id', 'addr', ]], how='left', on='ad_id')

In [187]:
df_dadata_houses_cian.to_csv('dadata_houses_cian_16_sample.csv')

In [157]:
# циан: 
# запрос к дадате 500 адресов - 2.5 минуты
# за день приходит 2700 записей по 7 городам
# в итоге время загрузки увеличивается на 2902/500 * 3 = 17 минуты

In [257]:
# сейчас в dadata_houses 5000 house_fias_id null на 119000 записей
# у меня 200/500

### загрузка

In [239]:
import json

In [246]:
ddt_avito = pd.read_csv('dadata_houses_avito_16_sample.csv', index_col=0)
ddt_cian = pd.read_csv('dadata_houses_cian_16_sample.csv', index_col=0)

In [247]:
def tojson(x):
    try:
        return json.loads(x.replace("'", '"').replace('None', 'null'))
    except:
        try:
            return json.loads(x.replace('"', "").replace("'", '"').replace('None', 'null'))
        except:
            return None

In [248]:
ddt_avito['data'] = ddt_avito['data'].apply(lambda x: tojson(x))

In [249]:
ddt_avito

,house_fias_id,data,geo_lat,geo_lon,street,house,qc,result,qc_geo,ad_id
0,93dd5940-a82b-4e15-b2f5-e3cbcd65e19e,{'source': 'Казань; с. Усады; Дорожная ул.; 69...,55.673752,49.217079,Дорожная,69,1,"Респ Татарстан, Лаишевский р-н, с Усады, ул До...",2,2302244851
1,c0c52465-8b8c-4d26-a027-f39c15051ba0,"{'source': 'Анапа; Анапское ш.; 30', 'result':...",44.898317,37.354429,Анапское,30,0,"Краснодарский край, г Анапа, ш Анапское, д 30",0,2160111672
2,NaN,{'source': 'Казань; ул. Кул Гали; ЖК «‎Изумруд...,55.750725,49.243161,Кул Гали,NaN,1,"г Казань, ул Кул Гали",2,2709454154
3,4ec12aab-20d3-4b7f-bc5e-ee729be0e529,{'source': 'Казань; р-н Ново-Савиновский; ул. ...,55.840791,49.146788,Адоратского,33,0,"г Казань, ул Адоратского, д 33",0,2954503264
4,432050a3-99bf-4441-a341-914b1fc79a7d,{'source': 'Краснодар; р-н Прикубанский; жилой...,45.104595,38.955110,Степная,1/1,0,"г Краснодар, тер им. Демьяна Бедного жилой мас...",0,2982556396
...,...,...,...,...,...,...,...,...,...,...
495,NaN,{'source': 'Краснодар; р-н Центральный; ул. Об...,44.998027,39.025687,Обрывная,NaN,1,"г Краснодар, ул Обрывная",2,2602021054
496,8034091c-17ed-4ffe-86a0-f97336db95d6,{'source': 'Краснодар; р-н Карасунский; ул. Ва...,44.989475,39.069290,им. Валерия Гассия,4/7,1,"г Краснодар, ул им. Валерия Гассия, д 4/7",1,2538472715
497,4abecbfe-cbe2-4c91-b43b-004f985ea730,{'source': 'Краснодар; р-н Карасунский; ул. им...,45.013931,39.092328,им. Мачуги В.Н.,166,1,"г Краснодар, ул им. Мачуги В.Н., д 166 к 2",1,2699999521
498,4abecbfe-cbe2-4c91-b43b-004f985ea730,{'source': 'Краснодар; р-н Карасунский; ул. им...,45.013931,39.092328,им. Мачуги В.Н.,166,1,"г Краснодар, ул им. Мачуги В.Н., д 166 к 2",1,2700786916


In [323]:
ddt_avito[~ddt_avito.house_fias_id.isna()]

,house_fias_id,data,geo_lat,geo_lon,street,house,qc,result,qc_geo,ad_id
0,93dd5940-a82b-4e15-b2f5-e3cbcd65e19e,{'source': 'Казань; с. Усады; Дорожная ул.; 69...,55.673752,49.217079,Дорожная,69,1,"Респ Татарстан, Лаишевский р-н, с Усады, ул До...",2,2302244851
1,c0c52465-8b8c-4d26-a027-f39c15051ba0,"{'source': 'Анапа; Анапское ш.; 30', 'result':...",44.898317,37.354429,Анапское,30,0,"Краснодарский край, г Анапа, ш Анапское, д 30",0,2160111672
3,4ec12aab-20d3-4b7f-bc5e-ee729be0e529,{'source': 'Казань; р-н Ново-Савиновский; ул. ...,55.840791,49.146788,Адоратского,33,0,"г Казань, ул Адоратского, д 33",0,2954503264
4,432050a3-99bf-4441-a341-914b1fc79a7d,{'source': 'Краснодар; р-н Прикубанский; жилой...,45.104595,38.955110,Степная,1/1,0,"г Краснодар, тер им. Демьяна Бедного жилой мас...",0,2982556396
5,4abecbfe-cbe2-4c91-b43b-004f985ea730,{'source': 'Краснодар; р-н Карасунский; ул. им...,45.013931,39.092328,им. Мачуги В.Н.,166,1,"г Краснодар, ул им. Мачуги В.Н., д 166 к 2",1,2412284578
...,...,...,...,...,...,...,...,...,...,...
493,1c2f294a-1e15-4687-9124-c833fe9ffb11,{'source': 'Казань; Аметьевская магистраль; д....,55.766925,49.171477,Аметьевская магистраль,18,0,"г Казань, ул Аметьевская магистраль, д 18 к 5",1,2719787283
496,8034091c-17ed-4ffe-86a0-f97336db95d6,{'source': 'Краснодар; р-н Карасунский; ул. Ва...,44.989475,39.069290,им. Валерия Гассия,4/7,1,"г Краснодар, ул им. Валерия Гассия, д 4/7",1,2538472715
497,4abecbfe-cbe2-4c91-b43b-004f985ea730,{'source': 'Краснодар; р-н Карасунский; ул. им...,45.013931,39.092328,им. Мачуги В.Н.,166,1,"г Краснодар, ул им. Мачуги В.Н., д 166 к 2",1,2699999521
498,4abecbfe-cbe2-4c91-b43b-004f985ea730,{'source': 'Краснодар; р-н Карасунский; ул. им...,45.013931,39.092328,им. Мачуги В.Н.,166,1,"г Краснодар, ул им. Мачуги В.Н., д 166 к 2",1,2700786916


In [261]:
# connect to preprod db
# ssh_host, ssh_port, ssh_username, ssh_password, \
# database_username, database_password, database_name, \
# localhost, localhost_port, table_name, ya_token, ya_api, ya_link = get_config_1()
# sql_server, sql_engine = get_sql_engine(ssh_host, ssh_port, ssh_username, ssh_password, localhost, 
#                                         localhost_port, database_username, database_password, database_name)

In [262]:
# ddt_avito.to_sql(name='dadata_houses', con=sql_engine, if_exists='append', chunksize=7000, method='multi', index=False)

In [317]:
def get_districts_from_house(df, engine):
    unique_fias = tuple(df.dropna(subset='house_fias_id').house_fias_id.unique())
    get_districts_query = \
        """select dh.house_fias_id, h.jkh_id, h.id as house_id, h.district_id 
            from dadata_houses dh left join houses h on h.dadata_houses_id = dh.id 
            where h.district_id is not null
            and dh.house_fias_id in {}""".format(unique_fias)
    try:
        con_obj = engine.connect()
        districts_db = pd.read_sql(text(get_districts_query), con=con_obj)
        con_obj.close()
        exc_code = None
    except Exception as exc:
        print(traceback.format_exc())
        districts_db = None
        exc_code = exc.code
    return districts_db, exc_code

In [318]:
districts_df, error_loading_districts_from_houses = get_districts_from_house(ddt_avito, sql_engine)

In [319]:
districts_df.drop_duplicates(subset=['house_fias_id', 'district_id'], inplace=True)

In [320]:
new_df = pd.merge(ddt_avito, districts_df, how='left', on='house_fias_id')

In [321]:
new_df

,house_fias_id,data,geo_lat,geo_lon,street,house,qc,result,qc_geo,ad_id,jkh_id,house_id,district_id
0,93dd5940-a82b-4e15-b2f5-e3cbcd65e19e,{'source': 'Казань; с. Усады; Дорожная ул.; 69...,55.673752,49.217079,Дорожная,69,1,"Респ Татарстан, Лаишевский р-н, с Усады, ул До...",2,2302244851,NaN,NaN,NaN
1,c0c52465-8b8c-4d26-a027-f39c15051ba0,"{'source': 'Анапа; Анапское ш.; 30', 'result':...",44.898317,37.354429,Анапское,30,0,"Краснодарский край, г Анапа, ш Анапское, д 30",0,2160111672,NaN,NaN,NaN
2,NaN,{'source': 'Казань; ул. Кул Гали; ЖК «‎Изумруд...,55.750725,49.243161,Кул Гали,NaN,1,"г Казань, ул Кул Гали",2,2709454154,NaN,NaN,NaN
3,4ec12aab-20d3-4b7f-bc5e-ee729be0e529,{'source': 'Казань; р-н Ново-Савиновский; ул. ...,55.840791,49.146788,Адоратского,33,0,"г Казань, ул Адоратского, д 33",0,2954503264,55461.0,12716.0,163.0
4,432050a3-99bf-4441-a341-914b1fc79a7d,{'source': 'Краснодар; р-н Прикубанский; жилой...,45.104595,38.955110,Степная,1/1,0,"г Краснодар, тер им. Демьяна Бедного жилой мас...",0,2982556396,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,NaN,{'source': 'Краснодар; р-н Центральный; ул. Об...,44.998027,39.025687,Обрывная,NaN,1,"г Краснодар, ул Обрывная",2,2602021054,NaN,NaN,NaN
496,8034091c-17ed-4ffe-86a0-f97336db95d6,{'source': 'Краснодар; р-н Карасунский; ул. Ва...,44.989475,39.069290,им. Валерия Гассия,4/7,1,"г Краснодар, ул им. Валерия Гассия, д 4/7",1,2538472715,NaN,75286.0,32.0
497,4abecbfe-cbe2-4c91-b43b-004f985ea730,{'source': 'Краснодар; р-н Карасунский; ул. им...,45.013931,39.092328,им. Мачуги В.Н.,166,1,"г Краснодар, ул им. Мачуги В.Н., д 166 к 2",1,2699999521,NaN,NaN,NaN
498,4abecbfe-cbe2-4c91-b43b-004f985ea730,{'source': 'Краснодар; р-н Карасунский; ул. им...,45.013931,39.092328,им. Мачуги В.Н.,166,1,"г Краснодар, ул им. Мачуги В.Н., д 166 к 2",1,2700786916,NaN,NaN,NaN


In [322]:
new_df[~new_df.district_id.isna()]

,house_fias_id,data,geo_lat,geo_lon,street,house,qc,result,qc_geo,ad_id,jkh_id,house_id,district_id
3,4ec12aab-20d3-4b7f-bc5e-ee729be0e529,{'source': 'Казань; р-н Ново-Савиновский; ул. ...,55.840791,49.146788,Адоратского,33,0,"г Казань, ул Адоратского, д 33",0,2954503264,55461.0,12716.0,163.0
8,a815b58a-c635-40a1-a125-63f08da387ad,{'source': 'Казань; Республика Татарстан; улиц...,55.759734,49.236589,Академика Завойского,11А,0,"г Казань, ул Академика Завойского, д 11А",0,2396558525,55637.0,13804.0,155.0
16,0eb165f8-b2e3-4c59-9cd1-53581a856293,{'source': 'Новороссийск; с. Цемдолина; ул. Ки...,44.749019,37.710501,Кипарисовая,6,0,"Краснодарский край, г Новороссийск, с Цемдолин...",0,2416219349,54025.0,11068.0,281.0
24,2a0f8076-6db6-4098-96bb-e46fae78b832,{'source': 'Казань; р-н Советский; ул. Анаса Т...,55.794358,49.111497,Анаса Тазетдинова,6,0,"г Казань, ул Анаса Тазетдинова, д 6",4,2619353372,NaN,79961.0,362.0
29,56cfa35b-7d01-4baa-9fa4-479082f3c6fa,{'source': 'Краснодар; р-н Прикубанский; ул. З...,45.091994,38.911231,Западный Обход,42,0,"г Краснодар, ул Западный Обход, д 42 к 8",0,2846867889,47571.0,3353.0,40.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
488,b24ad300-49db-464f-8606-6b90a0b569f0,"{'source': 'Казань; ул. Аделя Кутуя; д. 3', 'r...",55.787277,49.166556,Аделя Кутуя,3,0,"г Казань, ул Аделя Кутуя, д 3",0,2266643334,55383.0,27224.0,362.0
489,6c1ba80f-09dd-414e-acdd-1e24d3d2cb19,{'source': 'Краснодар; р-н Центральный; Садова...,45.041910,38.993926,Садовая,159,3,"г Краснодар, ул Садовая, д 159 к 2",0,2580980829,52273.0,4800.0,36.0
490,dee5ca6a-4c73-4c2b-88a6-9b2a36e01b3e,{'source': 'Краснодар; р-н Карасунский; ул. Ур...,45.036351,39.070530,Уральская,87/7,0,"г Краснодар, ул Уральская, д 87/7",1,2538419583,NaN,75387.0,46.0
493,1c2f294a-1e15-4687-9124-c833fe9ffb11,{'source': 'Казань; Аметьевская магистраль; д....,55.766925,49.171477,Аметьевская магистраль,18,0,"г Казань, ул Аметьевская магистраль, д 18 к 5",1,2719787283,NaN,79908.0,362.0


In [ ]:
# получилось определить district у 148/500